## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


In [2]:
tf.__version__

'2.2.0'

In [3]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

## Optimization with Adam and original function

In [4]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    delta = tf.Variable(100., trainable = True)
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(80000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [5]:

PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
losses = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    losses.append(loss.numpy())
    print("Data index {}. Delta {:.2f}. Loss {:.2f}".format(index_, delta_r, loss.numpy()))

  0%|          | 0/100 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
np.mean(losses)

In [ ]:
data = asarray(delta_seq)
save('delta.npy', delta_seq)

## Optimization with list of delta

In [4]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return tf.reduce_mean(influence)

def optimization(data):
    threhold = tf.constant([0.001])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0.0, 
        stddev=1.0, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = np.mean(data['mu1'] - data['mu0'])
    
    for i in range(50000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            loss = tf.math.abs(true_effect - influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            ## early stopping
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

### Optimize in train data

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(1, 101)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

  1%|          | 1/100 [00:05<08:26,  5.11s/it]

The performance reach MAE: 0.001. Cancelling the training at step 40
Data index 1. Loss 0.00


  2%|▏         | 2/100 [04:49<2:25:22, 89.01s/it]

Data index 2. Loss 0.04


  3%|▎         | 3/100 [09:32<3:57:55, 147.17s/it]

...

The performance reach MAE: 0.001. Cancelling the training at step 14
Data index 22. Loss 0.00


 23%|██▎       | 23/100 [1:35:35<4:59:42, 233.54s/it]

Data index 23. Loss 0.05


 24%|██▍       | 24/100 [1:40:25<5:17:21, 250.55s/it]

Data index 24. Loss 0.01


In [ ]:
save('list_delta_.npy', delta_seq)






## Optimization for individual treatment effect

In [ ]:
def incre_ps(delta, data):
    q1 = (delta * data['p1']) / (delta * data['p1'] + data['p0'])
    q1 = tf.math.abs(q1)
    a0 = (1-q1)*data['w0']*(data['cf0'] - data[outcome])
    a1 = q1*data['w1']*(data['cf1'] - data[outcome])    
    influence = a1 - a0
    return influence

def optimization(data):
    threhold = tf.constant([0.01])
    '''
    delta = tf.Variable(
        tf.random.uniform([data.shape[0],], 
                          minval=1, 
                          maxval=100, 
                          dtype=tf.dtypes.float32), 
                          trainable = True)
    '''
    delta = tf.Variable(tf.random.normal(
        [data.shape[0],], 
        mean=0, 
        stddev=10, 
        dtype=tf.dtypes.float32, 
        seed=1, 
        name='delta'
    ), trainable = True)
    
    true_effect = data['mu1'] - data['mu0']
    
    for i in range(100000):
        with tf.GradientTape() as tape:
            influence = incre_ps(delta, data)
            #print(true_effect)
            #print(influence)
            loss = tf.keras.losses.MAE(true_effect, influence)
            d_delta = tape.gradient(loss, delta)
            opt = tf.keras.optimizers.Adam(learning_rate=0.001)
            opt.apply_gradients(zip([d_delta], [delta]))
            if i % 1000 == 0:
                print("Epoch {}. Loss {:.5f}".format(i, loss.numpy()))
            if tf.math.less(loss, threhold):
                print("The performance reach MAE: 0.001. Cancelling the training at step {}".format(i))
                break
    return delta, loss

In [ ]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

delta_seq = []
for index_ in tqdm(range(13, 14)): 
    data = utils.load_data(PATH_TRAIN, index_)
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    data['p1'] = model_t.predict_proba(data[cov])[:,1]
    data['p0'] = 1 - data['p1']

    ## Compute counterfactual outcome with no treatment
    data_pos = data.copy()
    data_pos[treatment] = 1
    data['cf1'] = model_y.predict(data_pos[features])

    ## Compute counterfactual outcome with treatment
    data_neg = data.copy()
    data_neg[treatment] = 0
    data['cf0'] = model_y.predict(data_neg[features])

    data['ips_weight'] = (data[treatment] / data['p1'] + (1 - data[treatment]) /
                          (1 - data['p1']))
    
    data['w0'] = data['ips_weight']*data[treatment]
    data['w1'] = data['ips_weight']*(1 - data[treatment])
    
    delta, loss = optimization(data)
    delta_r = delta.numpy()
    delta_seq.append(delta_r)
    print("Data index {}. Loss {:.2f}".format(index_, loss.numpy()))

In [12]:
save('individual_list_delta_.npy', delta_seq)